# Objectif de ce notebook
1. Modeliser le comportement des items dans Dofus, les caracteristiques des items et leurs limites
2. Introduire les notions de forgemagie et la modification des caractéristiques initiales
3. Générer aléatoirement des patternes d'item avec les valeurs min, max des stats de l'item
4. Générer des items à partir patternes d'item avec des stats réelles
5. Evaluer ces items selon un algorithme de base
6. Remplir un dataframe avec ces items et en faire un fichier.csv que l'on va étudier et soumettre à un modèle de ML pour prédire la qualité

# Importer toutes les libraries nécessaires avec la cellule suivante

In [1]:
import numpy as np
import pandas as pd
import random
import string

# ****1. Définition d'un item et de ses caractéristiques****
## Objectifs
- Définir ce qu'est un item, ses caractéristiques, les valeures théoriques maximales et minimales
- Créer une classe ItemTemplate qui décrit un patterne d'item selon ses valeures théoriques
- Créer une classe ItemInstance qui définit les valeures réelles de l'item
- Stocker chaque type de caractéristique du jeu dans un dictionnaire
- Créer des fonctions qui évaluent mon item de façon brève afin de simuler une récupération de données manuellement
- Créer une fontion qui peut générer des items aléatoirement
## 1.1. Définition de la classe StatDefinition
Définit une statistique de base d'un item, avec sa valeur minimale, maximale théorique

In [2]:
class StatDefinition:
    def __init__(self, name, min_value, max_value, type="fixe", weight=1):
        self.name = name
        self.min_value = min_value
        self.max_value = max_value
        self.type = type
        self.weight = weight

    def __repr__(self):
        return f"{self.name} ({self.min_value}-{self.max_value}, poids={self.weight})"

## 1.2. Dictionnaire stat_pool
Dictionnaire des statistiques du jeu

In [3]:
stat_pool = {
    "vitalité": StatDefinition("vitalité", 0, 505, weight=0.2),
    "sagesse": StatDefinition("sagesse", 0, 60, weight=3),
    "force": StatDefinition("force", 0, 101, weight=1),
    "chance": StatDefinition("chance", 0, 101, weight=1),
    "agilité": StatDefinition("agilité", 0, 101, weight=1),
    "intelligence": StatDefinition("intelligence", 0, 101, weight=1),
    "dommage": StatDefinition("dommages", 0, 20, weight=20),
    "dommage_neutre" : StatDefinition("dommage_neutre", 0, 20, weight = 5),
    "dommage_terre" : StatDefinition("dommage_terre", 0, 20, weight = 5),
    "dommage_feu" : StatDefinition("dommage_feu", 0, 20, weight = 5),
    "dommage_eau" : StatDefinition("dommage_eau", 0, 20, weight = 5),
    "dommage_air" : StatDefinition("dommage_air", 0, 20, weight = 5),
    "dommage_critique" : StatDefinition("dommage_critique", 0, 25, weight = 5),
    "puissance": StatDefinition("puissance", 0, 100, weight=2),
    "coups_critiques": StatDefinition("coups_critiques", 0, 10, weight=10),
    "invocation": StatDefinition("invocation", 0, 2, weight=30),
    "résistance_neutre": StatDefinition("résistance_neutre", 0, 25, weight=2),
    "résistance_terre": StatDefinition("résistance_terre", 0, 25, weight=2),
    "résistance_feu": StatDefinition("résistance_feu", 0, 25, weight=2),
    "résistance_eau": StatDefinition("résistance_eau", 0, 25, weight=2),
    "résistance_air": StatDefinition("résistance_air", 0, 25, weight=2),
    "résistance_poussée": StatDefinition("résistance_poussée", 0, 80, weight=2),
    "résistance_critique": StatDefinition("résistance_critique", 0, 40, weight=2),
    "résistance_neutre_%": StatDefinition("résistance_neutre_%", 0, 15, weight=6),
    "résistance_terre_%": StatDefinition("résistance_terre_%", 0, 15, weight=6),
    "résistance_feu_%": StatDefinition("résistance_feu_%", 0, 15, weight=6),
    "résistance_eau_%": StatDefinition("résistance_eau_%", 0, 15, weight=6),
    "résistance_air_%": StatDefinition("résistance_air_%", 0, 15, weight=6),
    "tacle": StatDefinition("tacle", 0, 20, weight=4),
    "fuite": StatDefinition("fuite", 0, 20, weight=4),
    "soin": StatDefinition("soins", 0, 30, weight=10),
    "PA": StatDefinition("PA", 0, 1, weight=100),
    "PM": StatDefinition("PM", 0, 1, weight=90),
    "PO": StatDefinition("PO", 0, 2, weight=51),
    "prospection": StatDefinition("prospection", 0, 33, weight=3),
    "initiative": StatDefinition("initiative", 0, 800, weight=0.1),
    "pods": StatDefinition("pods", 0, 2000, weight=0.25),
    "dommage_dist_%" : StatDefinition("dommage_dist_%", 0, 5, weight=15),
    "dommage_sort_%" : StatDefinition("dommage_sort_%", 0, 5, weight=15),
    "dommage_mêlee_%" : StatDefinition("dommage_mêlee_%", 0, 5, weight=15),
    "résistance_dist_%" : StatDefinition("résistance_dist_%", 0, 5, weight=15),
    "résistance_sort_%" : StatDefinition("résistance_sort_%", 0, 5, weight=15),
    "résistance_melee_%" : StatDefinition("résistance_melee_%", 0, 5, weight=15)
}

essential_stats = {
    "PA", "PM", "PO", "invocation",
    "coups_critiques",
    "résistance_dist_%", "résistance_sort_%", "résistance_mêlée_%", 
    "dommage_mêlée_%", "dommage_dist_%", "dommage_sort_%"
}

basic_stats = {
    "vitalité", "force", "chance", "agilité", "intelligence",
    "puissance"
}

secondary_stats = {
    "sagesse", "tacle", "fuite", "prospection",
    "initiative", "pods", "soin"
}

heavy_stats = {
    "PA", "PM", "PO", "invocation",
    "sagesse", "coup_critiques", 
    "résistance_dist_%", "résistance_sort_%", "résistance_mêlée_%", 
    "dommage_mêlée_%", "dommage_dist_%", "dommage_sort_%"
}

résistance_pourcent_stats = {
    "résistance_neutre_%", "résistance_terre_%", "résistance_feu_%", "résistance_eau_%", "résistance_air_%",
    "résistance_dist_%", "résistance_sort_%", "résistance_mêlée_%", 
}

résistance_elem_stats = {
    "résistance_neutre", "résistance_terre", "résistance_feu", "résistance_eau", "résistance_air", 
    "résistance_poussée", "résistance_critique"
}

dommage_elem_stats = {
    "dommage_neutre", "dommage_terre", "dommage_feu", "dommage_eau", "dommage_air", 
    "dommage_critique", "dommage_poussée"
}

dommage_pourcent_stats = {
    "dommage_mêlée_%", "dommage_dist_%", "dommage_sort_%"
}

## 1.3. Classe ItemTemplate
Sert à générer un patterne d'item, les statistiques de base d'un item, jets théoriques min et max

In [4]:
class ItemTemplate:
    def __init__(self, name, stats, pui_category="moyen"):
        self.name = name
        self.stats = stats
        self.pui_category = pui_category

    def get_stat_max(self, stat_name):
        return self.stats.get(stat_name).max_value if stat_name in self.stats else 0

    def get_stat_weight(self, stat_name):
        return self.stats.get(stat_name).weight if stat_name in self.stats else 0

    def has_stat(self, stat_name):
        return stat_name in self.stats

## 1.4. Class ItemInstance
Définit un Item avec ses statistiques réelles en tenant compte de celles du patterne

In [5]:
class ItemInstance:
    def __init__(self, template, current_stats, label=None):
        self.template = template
        self.current_stats = current_stats
        self.label = label

    def get_ratio(self, stat):
        base = self.template.get_stat_max(stat)
        val = self.current_stats.get(stat, 0)
        if base <= 0:
            return np.nan
        return val / base

    def is_over(self, stat):
        ratio = self.get_ratio(stat)
        return ratio > 1 if not np.isnan(ratio) else False

    def is_exo(self, stat):
        return stat not in self.template.stats and self.current_stats.get(stat, 0) > 0

    def get_total_weight(self):
        weight = 0
        for stat in self.template.stats:
            value = min(self.current_stats.get(stat, 0), self.template.get_stat_max(stat))
            weight += value * self.template.get_stat_weight(stat)
        return weight

    def get_exo_weight(self):
        weight = 0
        for stat in self.current_stats:
            if self.is_exo(stat) and stat in stat_pool:
                weight += self.current_stats[stat] * stat_pool[stat].weight
        return weight

    def get_stat_tolerance(stat_name, base_value, current_value, weight_unit):
        stat_weight = current_value * weight_unit
        if stat_weight <= 30 * weight_unit:
            return base_value
        elif stat_weight <= 60 * weight_unit:
            return 3 * base_value
        else:
            return 10 * base_value

    def get_features(self):
        nb_stats = len(self.template.stats)
        perfect_lines = sum(
            1 for stat in self.template.stats
            if self.current_stats.get(stat, 0) == self.template.get_stat_max(stat)
        )
        high_ratios = sum(
            1 for stat in self.template.stats
            if 0.9 < self.get_ratio(stat) < 1
        )
        nb_overs = sum(1 for stat in self.template.stats if self.is_over(stat))
        total_weight = self.get_total_weight()
        exo_weight = self.get_exo_weight()
        avg_ratio = np.nanmean([self.get_ratio(stat) for stat in self.template.stats])
        over_weight = sum(
            (self.current_stats[stat] - self.template.get_stat_max(stat)) * self.template.get_stat_weight(stat)
            for stat in self.template.stats
            if self.is_over(stat)
        )

        nb_basic_stats_ratio = sum(1 for stat in self.template.stats if stat in basic_stats) / nb_stats if nb_stats else 0
        nb_essential_stats_ratio = sum(1 for stat in self.template.stats if stat in essential_stats) / nb_stats if nb_stats else 0
        nb_secondary_stats_ratio = sum(1 for stat in self.template.stats if stat in secondary_stats) / nb_stats if nb_stats else 0
        nb_heavy_stats_ratio = sum(1 for stat in self.template.stats if stat in heavy_stats) / nb_stats if nb_stats else 0
        nb_résistance_elem_ratio = sum(1 for stat in self.template.stats if stat in résistance_elem_stats) / nb_stats if nb_stats else 0
        nb_resistance_pourcent_ratio = sum(1 for stat in self.template.stats if stat in résistance_pourcent_stats) / nb_stats if nb_stats else 0
        nb_dommage_elem_ratio = sum(1 for stat in self.template.stats if stat in dommage_elem_stats) / nb_stats if nb_stats else 0
        nb_dommage_pourcent_ratio = sum(1 for stat in self.template.stats if stat in dommage_pourcent_stats) / nb_stats if nb_stats else 0

        return {
            "item_name": self.template.name,
            "pui_category": self.template.pui_category,
            "nb_stats": nb_stats,
            "nb_perfect_lines": perfect_lines,
            "nb_high_ratio": high_ratios,
            "nb_overs": nb_overs,
            "total_weight": total_weight,
            "exo_weight": exo_weight,
            "over_weight": over_weight,
            "avg_ratio": avg_ratio,
            "is_exo": exo_weight > 0,
            "is_over": nb_overs > 0,
            "nb_basic_stats_ratio": nb_basic_stats_ratio,
            "nb_essential_stats_ratio": nb_essential_stats_ratio,
            "nb_secondary_stats_ratio": nb_secondary_stats_ratio,
            "nb_heavy_stats_ratio": nb_heavy_stats_ratio,
            "nb_résistance_elem_ratio": nb_résistance_elem_ratio,
            "nb_resistance_pourcent_ratio": nb_resistance_pourcent_ratio,
            "nb_dommage_elem_ratio": nb_dommage_elem_ratio,
            "nb_dommage_pourcent_ratio": nb_dommage_pourcent_ratio
        }

    def evaluate_quality_algo(self):
        def stat_is_perfect(stat):
            return self.current_stats.get(stat, 0) == self.template.get_stat_max(stat)

        def stat_is_within_tolerance(stat):
            if stat not in stat_pool:
                return True
            max_val = self.template.get_stat_max(stat)
            val = self.current_stats.get(stat, 0)
            base_weight = stat_pool[stat].weight
            if max_val == 0:
                return True
            diff = abs(max_val - val)
            line_weight = max_val * base_weight
            if line_weight <= 30:
                return diff <= 1
            elif line_weight <= 60:
                return diff <= 3
            else:
                return diff <= 10

        stats = self.template.stats
        n_stats = len(stats)

        perfect_lines = sum(stat_is_perfect(stat) for stat in stats)
        high_ratios = sum(1 for stat in stats if 0.9 <= self.get_ratio(stat) < 1)
        ratio_perfect = perfect_lines / n_stats if n_stats else 0

        total_weight = self.get_total_weight()
        exo_weight = self.get_exo_weight()
        over = any(self.is_over(stat) for stat in stats)
        exo = exo_weight > 0

        pui = self.template.pui_category
        note = 100
        malus = 0

        # Malus sur les essential stats non parfaites
        for stat in essential_stats:
            if stat in stats and self.get_ratio(stat) < 1:
                malus += 20

        # Malus si résistances pas parfaites
        for stat in résistance_elem_stats.union(résistance_pourcent_stats):
            if stat in stats and self.get_ratio(stat) < 1:
                malus += 10

        # Tolérances pour les autres lignes
        for stat in stats:
            if stat in essential_stats or stat not in stat_pool:
                continue
            if not stat_is_within_tolerance(stat):
                malus += 5

        # Bonus si très bon jet
        if malus < 10 and ratio_perfect > 0.8:
            note = 100
            quality = "parfait"
        elif malus < 25 and all(
            (stat not in stats or self.get_ratio(stat) == 1)
            for stat in essential_stats.union(résistance_pourcent_stats, résistance_elem_stats)
        ):
            note = max(85, 100 - malus)
            quality = "très bon jet"
        elif all(self.get_ratio(stat) >= 0.6 for stat in stats if stat in stat_pool):
            note = max(60, 100 - malus)
            quality = "jet craft"
        else:
            note = max(30, 100 - malus)
            quality = "jet nul"

        return {
            "note": note,
            "exo": exo,
            "over": over,
            "puit": pui,
            "quality": quality
        }

## 1.5. Fonction classify_pui(stats)
Fonction qui classify les items selon leur poids de base

In [6]:
def classify_pui(stats):
    if any(stat in stats for stat in ["PA", "PM", "PO"]):
        return "grand"
    if ("invocation" in stats or
        ("sagesse" in stats and stats["sagesse"].max_value > 40) or
        ("coups_critiques" in stats and stats["coups_critiques"].max_value > 4) or
        any(stat.startswith("dommage") for stat in stats)):
        return "moyen"
    return "faible"

## 1.6. Fonction generate_random_template()
Fonction qui génère un template d'item théorique

In [7]:
def generate_random_template():
    name = "Item_" + ''.join(random.choices(string.ascii_uppercase, k=5))
    stats = {}

    # Stat commune : sagesse (90% de chance)
    if random.random() < 0.9:
        definition = stat_pool["sagesse"]
        stats["sagesse"] = StatDefinition("sagesse", 20, 60, weight=definition.weight)

    # Stat très fréquente : vitalité
    definition = stat_pool["vitalité"]
    stats["vitalité"] = StatDefinition("vitalité", 100, 400, weight=definition.weight)

    # Coups critiques (1 chance sur 2, max 7)
    if random.random() < 0.5:
        definition = stat_pool["coups_critiques"]
        stats["coups_critiques"] = StatDefinition("coups_critiques", 1, 7, weight=definition.weight)

    # Stats élémentaires (90% de chance d'en avoir au moins une)
    elem_stats = ["force", "intelligence", "chance", "agilité"]
    if random.random() < 0.9:
        selected_elem = random.sample(elem_stats, k=random.choice([1, 2]))
        for elem in selected_elem:
            definition = stat_pool[elem]
            max_val = random.choice([v for v in range(30, definition.max_value + 1, 5)])
            min_val = max(5, int(max_val * 0.5))
            min_val -= min_val % 5
            stats[elem] = StatDefinition(elem, min_val, max_val, weight=definition.weight)

            # Associer une stat de dommage à l'élément
            elem_dmg = f"dommage_{elem}" if f"dommage_{elem}" in stat_pool else "dommage"
            if elem_dmg in stat_pool:
                dmg_def = stat_pool[elem_dmg]
                stats[elem_dmg] = StatDefinition(elem_dmg, 3, dmg_def.max_value, weight=dmg_def.weight)

    # Résistances (1 chance sur 2 d'en avoir une)
    resistance_stats = [s for s in stat_pool if "résistance" in s and s not in stats]
    if random.random() < 0.5 and resistance_stats:
        stat = random.choice(resistance_stats)
        definition = stat_pool[stat]
        stats[stat] = StatDefinition(stat, 3, definition.max_value, weight=definition.weight)

    # Ajout aléatoire de 1 à 3 autres stats restantes
    remaining = [s for s in stat_pool if s not in stats and stat_pool[s].max_value > 0]
    for stat in random.sample(remaining, k=random.randint(1, 3)):
        definition = stat_pool[stat]
        max_cap = definition.max_value
        if max_cap >= 30:
            max_val = random.choice([v for v in range(30, max_cap + 1, 5)])
        else:
            max_val = random.randint(1, max_cap)
        min_val = max(5, int(max_val * 0.5))
        min_val -= min_val % 5
        stats[stat] = StatDefinition(stat, min_val, max_val, weight=definition.weight)

    # Détermination automatique du puit
    if any(stat in stats for stat in ["PA", "PM", "PO"]):
        pui_category = "gros"
    elif any(stat in stats for stat in ["invocations", "coups_critiques"]) or ("sagesse" in stats and stats["sagesse"].max_value >= 40):
        pui_category = "moyen"
    else:
        pui_category = "faible"

    return ItemTemplate(name, stats, pui_category=pui_category)

In [8]:
def generate_perfect_instance(template):
    current_stats = {}

    for stat, definition in template.stats.items():
        # Stats essentielles, résistances et dommages : on met max directement
        if stat in essential_stats or "résistance" in stat or "dommage" in stat:
            current_stats[stat] = definition.max_value

        # Autres stats : proche du max, mais sans over
        else:
            current_stats[stat] = random.choice([
                definition.max_value,
                max(definition.max_value - 1, definition.min_value)
            ])

    # Aucun exo ou over
    instance = ItemInstance(template, current_stats)
    return instance


In [9]:
def generate_high_quality_instance(template):
    current_stats = {}

    for stat, definition in template.stats.items():
        if stat in essential_stats or "résistance" in stat:
            current_stats[stat] = definition.max_value
        else:
            base = definition.max_value
            current_stats[stat] = random.randint(int(0.9 * base), base)

    instance = ItemInstance(template, current_stats)
    return instance


In [10]:
def generate_bad_instance(template):
    current_stats = {}
    for stat, definition in template.stats.items():
        # Valeur très basse volontairement
        val = random.randint(definition.min_value, max(definition.min_value + 1, int(definition.max_value * 0.5)))
        current_stats[stat] = val

    # Aucun exo ni over
    return ItemInstance(template, current_stats)


## 1.7. Fonction generate_random_instance(template)
Fonction qui génère aléatoirement des instances d'items à partir d'un template

In [11]:
def generate_random_instance(template, debug=False):
    exo_types = []
    current_stats = {}
    elemental_stats = {"force", "intelligence", "chance", "agilité", "sagesse", "vitalité"}

    # Stats de base
    for stat, definition in template.stats.items():
        if stat in elemental_stats:
            val = random.choice([v for v in range(int(definition.max_value * 0.6), definition.max_value + 1, 5)])
        elif stat in {"coups_critiques", "PA", "PM", "PO", "invocation"} or "résistance_" in stat:
            val = definition.max_value if random.random() < 0.8 else random.randint(int(definition.max_value * 0.6), definition.max_value)
        elif "dommage" in stat:
            if definition.max_value > 11:
                val = random.choice([definition.max_value - 1, definition.max_value]) if random.random() < 0.8 else random.randint(int(definition.max_value * 0.6), definition.max_value - 2)
            else:
                val = random.randint(int(definition.max_value * 0.6), definition.max_value)
        else:
            val = random.randint(int(definition.max_value * 0.6), definition.max_value)
        current_stats[stat] = val

    # Déterminer le puit
    if any(stat in template.stats for stat in ["PA", "PM", "PO"]):
        puit = "gros"
        max_exo_weight = 101
        weight_limit = 100
        exo_candidates = ["PA", "PM", "PO"] + [s for s in stat_pool if s not in template.stats and stat_pool[s].weight <= 50]
    elif any(stat in template.stats for stat in ["invocation", "coups_critiques"]) or ("sagesse" in template.stats and template.get_stat_max("sagesse") >= 40):
        puit = "moyen"
        max_exo_weight = 50
        weight_limit = 50
        exo_candidates = [s for s in stat_pool if s not in template.stats and stat_pool[s].weight <= 30]
    else:
        puit = "faible"
        max_exo_weight = 20
        weight_limit = 20
        exo_candidates = [s for s in stat_pool if s not in template.stats and stat_pool[s].weight <= 15]

    total_exo_weight = 0
    random.shuffle(exo_candidates)

    for exo_stat in exo_candidates:
        if exo_stat not in stat_pool:
            continue
        definition = stat_pool[exo_stat]
        is_major_exo = exo_stat in {"PA", "PM", "PO"}
        chance = 0.5 if not is_major_exo and not exo_types else 0.01
        if random.random() > chance:
            continue

        exo_val = min(random.randint(1, definition.max_value), 10)  # on limite la valeur brute
        added_weight = exo_val * definition.weight

        if total_exo_weight + added_weight <= max_exo_weight:
            current_stats[exo_stat] = exo_val
            exo_types.append("majeur" if is_major_exo else "classique")
            total_exo_weight += added_weight
            if debug:
                print(f"[DEBUG] Ajout exo {exo_stat} = {exo_val} (poids: {added_weight}, total: {total_exo_weight})")

    instance = ItemInstance(template, current_stats)
    instance.exo_types = exo_types
    return instance

# ****2. Création d'un jeu de données artificielles****
## 2.1. Fonction generate_dataset(nb_templates, nb_instance_per_case)
Création d'un dataset selon le nombre de templates, et nombre d'instances par template

In [12]:
def generate_dataset(nb_templates=100, nb_instances_per_quality=50):
    dataset = []
    templates = [generate_random_template() for _ in range(nb_templates)]

    for template in templates:
        for quality in ["parfait", "très bon jet", "jet craft", "jet nul"]:
            for _ in range(nb_instances_per_quality):
                if quality == "parfait":
                    instance = generate_perfect_instance(template)
                elif quality == "très bon jet":
                    instance = generate_high_quality_instance(template)
                elif quality == "jet craft":
                    instance = generate_random_instance(template)
                elif quality == "jet nul":
                    instance = generate_bad_instance(template)
                else:
                    continue  # sécurité

                # Extraire toutes les features + l'évaluation
                features = instance.get_features()
                eval_quality = instance.evaluate_quality_algo()

                features.update(eval_quality)
                features["template_name"] = template.name
                features["config_case"] = quality
                dataset.append(features)

    return pd.DataFrame(dataset)


## 2.2. Génération du dataset
- Une fois généré ne plus exécuter ou commenter
- Si dataset pas assez pertinent, regénerer et refaire les tests (voir partie 3)

In [15]:
import os

dataset = generate_dataset()

# Détection automatique de l’environnement Kaggle
running_on_kaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE") is not None

if running_on_kaggle:
    # Environnement Kaggle : enregistrer à la racine (pour apparaître dans "Output")
    dataset.to_csv("item_dataset_2.csv", index=False)
else:
    # Environnement local : dossier structuré
    os.makedirs("../data/processed", exist_ok=True)
    dataset.to_csv("../data/processed/item_dataset_4.csv", index=False)